In [1]:
import json
import random
from dotenv import load_dotenv
import psycopg2
import os

# load the seed file
with open('seed.json', encoding='utf-8') as f:
    seed = json.load(f)

In [2]:
def default_programs():
    tutor_img_num, mentor_img_num = random.sample(range(1, 6), 2)
    event_img_num = random.randint(1,10)
    return [
        {
            "name": "Tutoring",
            "details": "Provide one-on-one academic support to students on a range of topics",
            "url": "",
            "img": f"/volunteer/tutoring/stock{tutor_img_num}.png",
            "category": "volunteer",
        },
        {
            "name": "Event Volunteer",
            "details": "Provide support for school-sponsored events",
            "url": "",
            "img": f"/volunteer/event/stock{event_img_num}.png",
            "category": "volunteer",
        },
        {
            "name": "Career Prep and Mentoring",
            "details": "Provide students with mentoring, career insight and readiness",
            "url": "",
            "img": f"/volunteer/event/stock{mentor_img_num}.png",
            "category": "volunteer",
        }
    ]

def extract_program_details(school):
    data = default_programs()
    for program in school["programs"]["createMany"]["data"]:
        if len(program["url"]):
            if "img" in program:
                data.append(
                    {
                        "name": program["name"],
                        "details": program["details"],
                        "url": program["url"],
                        "img": program["img"],
                        "category": program["category"],
                    }
                )
            else:
                data.append(
                    {
                        "name": program["name"],
                        "details": program["details"],
                        "url": program["url"],
                        "category": program["category"],
                    }
                )
    # return [ f"{'*' if len(program['url']) else ''}{program['name']} ({program['category']}){'#' if has_kw(program['name']) else ''}" for program in school['programs']['createMany']['data'] if len(program['url'])]
    return data

# correct programs
for school in seed:
    school["programs"]["createMany"]["data"] = extract_program_details(school)


In [ ]:
def extract_profile(school):
    generated = school["generatedProfile"]
    address = school["geolocations"][0]["addressString"]
    retval = {
        "about": generated["overview"],
        "about_bp": generated["bullets"],
        "principal": school["principal"],
        "website_url": school["schoolUrl"],
        "donation_text": f"Mail a check made out to {school['schoolLabel']} and mail it to:\n{school['schoolLabel']}\n{address}\n",
        "volunteer_form_url": f"",
    }

    if "ytCodes" in school and len(school["ytCodes"]):
        retval["noteable_video"] = f'https://www.youtube.com/embed/{school["ytCodes"][0]}?autoplay=0&start=0&rel=0'

    return retval
    
    
def to_seed_entry(school):
    addr = school["geolocations"][0]
    addr_details = addr["addressDetails"]
    longitude, latitude = addr["geo"]
    
    retval = {
        "casid": school["casid"],
        "name": school["schoolLabel"],
        "address":addr["addressString"],
        "neighborhood": addr_details["District"],
        "priority": False,
        "latitude": str(latitude),
        "longitude": str(longitude),
        "profile": {"create": extract_profile(school)},
        "metrics": {"createMany": { "data": [], "skipDuplicates": True}},
        "programs": {"createMany": {"data": default_programs()}},
    }

    if "image" in school and school["image"] is not None:
        retval["img"] = school["image"]["filePath"]
    if "logo" in school and school["logo"] is not None:
        retval["logo"] = school["logo"]["filePath"]

    return retval

school_map = {school["casid"]: school for school in seed}
with open("schoolList_corrected.json", "r", encoding="utf-8") as f:
    school_list = json.load(f)
seed_list = [to_seed_entry(school) for school in school_list]

for seed_school in seed_list:
    if seed_school["casid"] not in school_map:
        school_map[seed_school["casid"]] = seed_school
    else:
        if "logo" in seed_school:
            school_map[seed_school["casid"]]["logo"] = seed_school["logo"]

print(json.dumps(school_map["3830387"]["metrics"], indent=4))

In [ ]:
def make_metrics(metrics, school_map):
    retval = []
    if metrics["enrolled"]:
        retval.append(
            {
                "name": "Students Enrolled",
                "value": metrics["enrolled"],
                "unit": "",
                "category": "about",
            }
        )
    if metrics["pct_met_and_above_english"]:
        retval.append(
            {
                "name": "English Proficiency",
                "value": int(metrics["pct_met_and_above_english"]),
                "unit": "%",
                "category": "outcome",
            }
        )
    if metrics["pct_met_and_above_math"]:
        retval.append(
            {
                "name": "Math Proficiency",
                "value": int(metrics["pct_met_and_above_math"]),
                "unit": "%",
                "category": "outcome",
            }
        )
    if metrics["pct_disabled"]:
        retval.append(
            {
                "name": "Students with Special Needs",
                "value": int(metrics["pct_disabled"] * 100.0),
                "unit": "%",
                "category": "about",
            }
        )
    if metrics["pct_elearners"]:
        retval.append(
            {
                "name": "English Language Learners",
                "value": int(metrics["pct_elearners"] * 100.0),
                "unit": "%",
                "category": "about",
            }
        )

    if metrics["school_code"] in school_map:
        school = school_map[metrics["school_code"]]
        lunch_metrics = [
            metric
            for metric in school["metrics"]["createMany"]["data"]
            if "lunch" in metric["name"].lower()
        ]
        if len(lunch_metrics) == 0:
            lunch_metrics = [
                {
                    "name": "Free/Reduced Lunch",
                    "value": int(metrics["pct_disadvantaged"]),
                    "unit": "%",
                    "category": "about",
                }
            ]
        retval.extend(lunch_metrics)

    return retval


# load metrics from db
load_dotenv()
try:
    conn = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST", "localhost"),
        port=os.getenv("DB_PORT", "5432"),
    )
    cur = conn.cursor()
    cur.execute(
        "SELECT school_code, enrolled, pct_met_and_above_english, pct_met_and_above_math, pct_disadvantaged, pct_elearners, pct_disabled FROM pivot_stats"
    )
    # data = [
    #     {
    #         "school_code": record[0],
    #         "enrolled": record[1],
    #         "pct_met_and_above_english": record[2],
    #         "pct_met_and_above_math": record[3],
    #         "pct_disadvantaged": record[4],
    #         "pct_elearners": record[5],
    #         "pct_disabled": record[6],
    #     }
    #     for record in cur.fetchall()
    # ]

    for record in cur.fetchall():
        data = {
            "school_code": record[0],
            "enrolled": record[1],
            "pct_met_and_above_english": record[2],
            "pct_met_and_above_math": record[3],
            "pct_disadvantaged": record[4],
            "pct_elearners": record[5],
            "pct_disabled": record[6],
        }
        if data["school_code"] in school_map:
            school_map[data["school_code"]]["metrics"] = {
                "createMany": {"data": make_metrics(data, school_map), "skipDuplicates": True}
            }

    cur.close()

    cur = conn.cursor()
    cur.execute(
        "SELECT school_code, rate_reg_hs_grads FROM grad_filtered WHERE rate_reg_hs_grads IS NOT NULL"
    )
    for record in cur.fetchall():
        data = {
            "school_code": record[0],
            "rate_reg_hs_grads": record[1],
        }
        if data["school_code"] in school_map:
            metrics = school_map[data["school_code"]]["metrics"]["createMany"]["data"]
            school_map[data["school_code"]]["metrics"]["createMany"]["data"] = [
                metric
                for metric in metrics
                if "graduation" not in metric["name"].lower()
            ]
            school_map[data["school_code"]]["metrics"]["createMany"]["data"].append(
                {
                    "name": "High School Graduation Rate",
                    "value": int(data["rate_reg_hs_grads"]),
                    "unit": "%",
                    "category": "outcome",
                }
            )

    cur.close()
    conn.close()
except psycopg2.DatabaseError as e:
    print(f"Database error: {e}")

# print(json.dumps(school_map["0119958"], indent=4))
# print(json.dumps(school_map["6102479"], indent=4))

with open("seed_scale.json", "w", encoding="utf-8") as f:
    json.dump(list(school_map.values()), f, indent=4)